# H2-O2 reaction inside a reactor cell

---

Chemical kinetics and thermodynamic calculations of gasous reactor cell analysis

In [1]:
import cantera as ct
import numpy as np, pandas as pd

In [2]:
gas = ct.Solution('../data/h2o2.yaml')

In [3]:
gas.TPX = 300.0, ct.one_atm, 'H2:2, O2:1'

In [4]:
gas.equilibrate('SV')

In [5]:
gas()


  ohmech:

       temperature   1530 K
          pressure   3.4454e+05 Pa
           density   0.48787 kg/m^3
  mean mol. weight   18.013 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.0665e+07       -1.9212e+08  J
   internal energy       -1.1372e+07       -2.0484e+08  J
           entropy             13403        2.4144e+05  J/K
    Gibbs function       -3.1173e+07       -5.6153e+08  J
 heat capacity c_p            2642.4             47598  J/K
 heat capacity c_v            2180.8             39284  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2        1.9272e-05         0.0001722           -26.103
                 H        9.9878e-09        1.7849e-07           -13.052
                 O        2.5716e-08        2.8954e-08           -18.0